# From Python to Production — Day 5  
## Notebook: NumPy in Depth — Arrays, Broadcasting & Vectorized Computing  
By **Prerna Joshi** | #25DaysOfDataTech

> *Vectorize your thinking — let arrays do the loops.*

**What you'll learn**
- Why NumPy is the backbone of data/ML stacks
- `ndarray` basics: creation, shapes, dtypes, indexing
- Views vs copies (and why it bites beginners)
- Broadcasting rules you’ll actually remember
- Vectorized math, ufuncs, aggregations, `axis` logic
- Boolean masks, fancy indexing, sorting & ranking
- Random numbers & reproducibility
- Linear algebra (`dot`, `matmul`, `einsum`), norms, SVD
- NaN-safe ops, numerical stability & memory layout
- Performance profiling: NumPy vs pure Python loops
- Mini projects: feature scaling, window ops, image-like transforms

> **Production mindset:** prefer vectorized ops, avoid hidden copies, measure before you micro-optimize, and write clear shape-aware code.

### 0. Setup

In [1]:
import numpy as np
np.__version__

'1.26.4'

### 1. Why NumPy?
NumPy provides a fast **N-dimensional array** object (`ndarray`) with efficient vectorized operations, which underpins pandas, scikit-learn, PyTorch/TensorFlow (at the edges), and many scientific libraries. It's the **language of shapes**.

### 2. ndarray Basics — Creation, Shape, Dtype

In [2]:
# Common ways to create arrays
a = np.array([1, 2, 3], dtype=np.int32)
b = np.arange(0, 10, 2)        # [0, 2, 4, 6, 8]
c = np.linspace(0, 1, 5)       # 5 points from 0 to 1 inclusive
d = np.zeros((2, 3))           # 2x3 of zeros (float64 by default)
e = np.ones((3, 2), dtype=np.float32)
f = np.eye(3)                  # Identity matrix
g = np.random.default_rng(42).normal(size=(2, 3))

a, a.shape, a.dtype, d.shape, e.dtype, g.mean().round(3)

(array([1, 2, 3]), (3,), dtype('int32'), (2, 3), dtype('float32'), -0.383)

**Shapes & reshaping**
- `shape` is a tuple like `(rows, cols, ...)`
- use `reshape`, `ravel`, `flatten` (note: `ravel` prefers a view), `np.newaxis` / `None` for adding dims.

In [3]:
x = np.arange(12)
x_3x4 = x.reshape(3, 4)         # (3,4)
x_row = x[np.newaxis, :]        # (1,12)
x_col = x[:, np.newaxis]        # (12,1)
x_3x4, x_row.shape, x_col.shape

(array([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]]),
 (1, 12),
 (12, 1))

### 3. Indexing & Slicing: The Shape-Aware Way
- **Basic slicing** returns views (when possible).
- **Fancy indexing** (using arrays/lists of indices) returns **copies**.
- **Boolean masks** filter data efficiently.

In [4]:
M = np.arange(1, 13).reshape(3, 4)
row2 = M[1]               # view of the 2nd row
col_last = M[:, -1]       # last column
sub = M[0:2, 1:3]         # 2x2 block (view)
fancy = M[[0, 2], [1, 3]] # picks (0,1) and (2,3) -> copy
mask = M % 2 == 0         # even elements (bool mask)
M, row2, col_last, sub, fancy, M[mask]

(array([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]]),
 array([5, 6, 7, 8]),
 array([ 4,  8, 12]),
 array([[2, 3],
        [6, 7]]),
 array([ 2, 12]),
 array([ 2,  4,  6,  8, 10, 12]))

### 4. Views vs Copies (Pitfall Alert)
Modifying a **view** changes the original; modifying a **copy** does not.

In [5]:
base = np.arange(6)
view = base[1:4]      # view
copy = base[[1, 3, 4]]# copy (fancy indexing)
view[:] = 99
copy[:] = -1
base, view, copy

(array([ 0, 99, 99, 99,  4,  5]), array([99, 99, 99]), array([-1, -1, -1]))

### 5. Dtypes, Numeric Precision & Memory
- Default dtype is often `float64`.
- Smaller dtypes save memory but may reduce precision.
- Use `astype` to convert.

In [6]:
arr64 = np.linspace(0, 1, 5, dtype=np.float64)
arr32 = arr64.astype(np.float32)
arr16 = arr64.astype(np.float16)
arr64.nbytes, arr32.nbytes, arr16.nbytes, arr16

(40, 20, 10, array([0.  , 0.25, 0.5 , 0.75, 1.  ], dtype=float16))

### 6. Broadcasting — Rules You Can Remember
Two dimensions are **compatible** if they are equal or one of them is **1**. Compare shapes from **right to left**.

Examples:
- `(3, 1)` with `(1, 4)` -> result `(3, 4)`
- `(5,)` with `(1, 5)` -> result `(1, 5)` (or `(5,)` depending on context)

In [7]:
A = np.arange(3).reshape(3, 1)       # (3,1): [[0],[1],[2]]
B = np.arange(4).reshape(1, 4)       # (1,4): [[0,1,2,3]]
A_plus_B = A + B                      # (3,4)
A_plus_B

array([[0, 1, 2, 3],
       [1, 2, 3, 4],
       [2, 3, 4, 5]])

### 7. Vectorized Math, Ufuncs & Aggregations
- Universal functions (ufuncs) operate elementwise (`np.add`, `np.exp`, `np.sqrt`).
- Aggregations like `sum`, `mean`, `std`, `min/max` can specify an `axis`.

In [8]:
Z = np.random.default_rng(0).normal(loc=10, scale=2, size=(4, 5))
elementwise = np.sqrt(Z.clip(min=0))   # safe sqrt
col_means = Z.mean(axis=0)             # per column
row_sums = Z.sum(axis=1)               # per row
overall = Z.mean()
elementwise.round(2), col_means.round(2), row_sums.round(2), round(overall, 2)

(array([[3.2 , 3.12, 3.36, 3.2 , 2.99],
        [3.27, 3.55, 3.45, 2.93, 2.73],
        [2.96, 3.18, 2.31, 3.09, 2.74],
        [2.92, 2.99, 3.06, 3.29, 3.48]]),
 array([ 9.57, 10.33,  9.47,  9.8 ,  9.  ]),
 array([50.41, 51.29, 41.26, 49.72]),
 9.63)

### 8. Boolean Logic, Masks & `where`

In [9]:
scores = np.array([88, 59, 73, 95, 41, 67])
grade = np.where(scores >= 90, "A",
         np.where(scores >= 75, "B",
         np.where(scores >= 60, "C", "D")))
passing = scores >= 60
scores[passing], grade

(array([88, 73, 95, 67]), array(['B', 'D', 'C', 'A', 'D', 'C'], dtype='<U1'))

### 9. Sorting, Ranking & Args
- `np.sort` returns a **sorted copy**; `arr.sort()` sorts **in-place**.
- `argsort` gives **indices** that would sort.

In [10]:
v = np.array([30, 10, 20, 10])
sorted_copy = np.sort(v)
idx = np.argsort(v, kind="stable")  # stable keeps order for ties
rank = np.empty_like(idx)
rank[idx] = np.arange(len(v))       # ranking from 0..n-1
v, sorted_copy, idx, rank

(array([30, 10, 20, 10]),
 array([10, 10, 20, 30]),
 array([1, 3, 2, 0], dtype=int64),
 array([3, 0, 2, 1], dtype=int64))

### 10. Random Numbers & Reproducibility
Prefer the modern `Generator` API: `np.random.default_rng(seed)`

In [11]:
rng = np.random.default_rng(123)
u = rng.uniform(0, 1, size=5)
n = rng.normal(0, 1, size=(2, 3))
choice = rng.choice(np.arange(10), size=5, replace=False)
u.round(3), n.round(3), choice

(array([0.682, 0.054, 0.22 , 0.184, 0.176]),
 array([[ 0.577, -0.636,  0.542],
        [-0.317, -0.322,  0.097]]),
 array([8, 1, 7, 9, 6]))

### 11. Linear Algebra — Dot, Matmul, Norms, SVD, `einsum`

In [12]:
X = np.random.default_rng(42).normal(size=(4, 3))
w = np.random.default_rng(0).normal(size=(3,))
y_pred = X @ w                 # same as np.matmul(X, w) or np.dot(X, w)

# Norms and SVD
fro = np.linalg.norm(X, ord='fro')
U, S, Vt = np.linalg.svd(X, full_matrices=False)

# einsum: sum over products with explicit index notation
eins = np.einsum('ij,j->i', X, w)  # same as X @ w

y_pred.round(3), round(fro, 3), S.round(3), np.allclose(y_pred, eins)

(array([ 0.656, -0.458,  0.047,  0.275]),
 3.217,
 array([2.963, 1.211, 0.321]),
 True)

### 12. NaNs & Numerical Stability
- Use `np.nan*` variants (`nanmean`, `nanstd`, …) when arrays may contain NaNs.
- For softmax/log-sum-exp, subtract max for stability.

In [13]:
data = np.array([1.0, np.nan, 3.0, 4.0, np.nan])
np.nanmean(data), np.nanstd(data)

(2.6666666666666665, 1.247219128924647)

In [14]:
# Numerically stable softmax
def softmax(x, axis=-1):
    x = np.asarray(x)
    x_shift = x - x.max(axis=axis, keepdims=True)
    ex = np.exp(x_shift)
    return ex / ex.sum(axis=axis, keepdims=True)

scores = np.array([12.0, 10.0, 8.0])
softmax(scores).round(6), (np.exp(scores)/np.exp(scores).sum()).round(6)

(array([0.866813, 0.11731 , 0.015876]), array([0.866813, 0.11731 , 0.015876]))

### 13. Memory Layout, Strides & Copies
- `C` vs `F` order affects performance when iterating along rows vs columns.
- `ravel(order='C'|'F')` for view-like flattening preference.

In [15]:
A = np.ascontiguousarray(np.arange(12).reshape(3,4))  # C-order
B = np.asfortranarray(np.arange(12).reshape(3,4))     # Fortran-order
A.flags['C_CONTIGUOUS'], B.flags['F_CONTIGUOUS']

(True, True)

### 14. Rolling/Windowed Ops (Stride Tricks — Advanced)

In [16]:
def rolling_window(a, window):
    # Creates a view of shape (n - w + 1, w)
    # NOTE: requires careful use; shapes must align
    from numpy.lib.stride_tricks import as_strided
    a = np.asarray(a)
    n = a.shape[0]
    if window > n:
        raise ValueError("window larger than array")
    return as_strided(a, shape=(n-window+1, window),
                      strides=(a.strides[0], a.strides[0]))

arr = np.arange(10)
rw = rolling_window(arr, 4)
rw, rw.mean(axis=1)

(array([[0, 1, 2, 3],
        [1, 2, 3, 4],
        [2, 3, 4, 5],
        [3, 4, 5, 6],
        [4, 5, 6, 7],
        [5, 6, 7, 8],
        [6, 7, 8, 9]]),
 array([1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5]))

### 15. Performance: Vectorization vs Python Loops
Measure with `%timeit` (run this in Jupyter). Here we emulate via Python.

In [17]:
# We'll do a simple benchmark inside Python (coarse)
import time

N = 1_000_00  # 100k
x = np.random.default_rng(0).random(N)

# Vectorized
t0 = time.time()
y_vec = x * 1.5 + 2
t1 = time.time()

# Pure Python loop
t2 = time.time()
y_loop = [xi * 1.5 + 2 for xi in x]  # list comp in CPython
t3 = time.time()

(t1 - t0, t3 - t2, np.allclose(y_vec, np.array(y_loop)))

(0.0010018348693847656, 0.03300762176513672, True)

### 16. Mini‑Project 1 — Feature Scaling & Z‑Scores (Fraud-ish vibes)
Given a 2D array of features, compute min‑max scaled and standardized versions, handling NaNs.

In [18]:
rng = np.random.default_rng(7)
X = rng.normal(loc=50, scale=10, size=(6, 4)).astype(float)
X[1, 2] = np.nan  # missing value

def minmax_scale(A):
    minv = np.nanmin(A, axis=0)
    maxv = np.nanmax(A, axis=0)
    return (A - minv) / (maxv - minv)

def zscore(A):
    mu = np.nanmean(A, axis=0)
    sigma = np.nanstd(A, axis=0)
    return (A - mu) / sigma

X_minmax = minmax_scale(X)
X_z = zscore(X)
np.round(X_minmax, 3), np.round(X_z, 3)

(array([[0.946, 1.   , 0.68 , 0.152],
        [0.712, 0.   ,   nan, 1.   ],
        [0.693, 0.288, 1.   , 0.626],
        [1.   , 0.047, 0.783, 0.755],
        [0.256, 0.414, 0.   , 0.   ],
        [0.   , 0.586, 0.265, 0.594]]),
 array([[ 0.958,  1.8  ,  0.372, -1.075],
        [ 0.308, -1.147,    nan,  1.394],
        [ 0.255, -0.299,  1.253,  0.306],
        [ 1.106, -1.007,  0.654,  0.68 ],
        [-0.959,  0.073, -1.505, -1.516],
        [-1.668,  0.581, -0.774,  0.211]]))

### 17. Mini‑Project 2 — Image‑like Transform (2D Convolution‑ish)
Apply a simple 3x3 mean filter to a 2D array using strides and broadcasting.

In [19]:
def mean_filter_3x3(img2d):
    img2d = np.asarray(img2d, dtype=float)
    H, W = img2d.shape
    # Extract 3x3 neighborhoods (valid)
    from numpy.lib.stride_tricks import as_strided
    shape = (H-2, W-2, 3, 3)
    strides = (img2d.strides[0], img2d.strides[1], img2d.strides[0], img2d.strides[1])
    patches = as_strided(img2d, shape=shape, strides=strides)
    return patches.mean(axis=(2,3))

img = np.arange(1, 26).reshape(5,5)
filtered = mean_filter_3x3(img)
img, filtered

(array([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10],
        [11, 12, 13, 14, 15],
        [16, 17, 18, 19, 20],
        [21, 22, 23, 24, 25]]),
 array([[ 7.,  8.,  9.],
        [12., 13., 14.],
        [17., 18., 19.]]))

### 18. Best Practices & Pitfalls
- Prefer vectorized ops; avoid Python loops for elementwise math.
- Be explicit about `axis` and shapes.
- Know when you're making a **copy** (fancy indexing) vs a **view** (basic slice).
- Use `default_rng(seed)` for reproducibility.
- Use NaN‑safe functions if data can be missing.
- Profile before optimizing; consider memory layout for large arrays.
- Document shape contracts in docstrings.

### 19. Exercises — Your Turn
1) **Broadcasting warm‑up**  
Create A of shape `(3,1)` and B of shape `(1,5)` filled with sequential ints. Compute `A*B + A` and verify the shape.

2) **Mask & Assign**  
Given `arr = np.array([5, -1, 7, 0, -3, 2])`, replace all negatives with their absolute values *in place* using a boolean mask.

3) **Columnwise Z‑Score**  
Write a function `col_zscore(M)` that standardizes each column of a 2D array (handle NaNs).

4) **Top‑k with argsort**  
For `v = rng.integers(0, 100, size=12)`, return the indices of the top‑3 values (stable for ties).

5) **Stable Softmax**  
Implement numerically stable softmax for a 2D array along `axis=1` and verify each row sums to 1.

### 20. Exercise Solutions
> Peek only after attempting!

In [45]:
# 1) Broadcasting warm-up
A = np.arange(3).reshape(3,1)
B = np.arange(5).reshape(1,5)
res = A*B + A
res.shape, res

((3, 5),
 array([[ 0,  0,  0,  0,  0],
        [ 1,  2,  3,  4,  5],
        [ 2,  4,  6,  8, 10]]))

In [46]:
# 2) Mask & Assign (in place)
arr = np.array([5, -1, 7, 0, -3, 2])
mask = arr < 0
arr[mask] = -arr[mask]
arr

array([5, 1, 7, 0, 3, 2])

In [47]:
# 3) Columnwise Z-Score with NaNs
def col_zscore(M):
    M = np.asarray(M, dtype=float)
    mu = np.nanmean(M, axis=0, keepdims=True)
    sd = np.nanstd(M, axis=0, keepdims=True)
    return (M - mu) / sd

M = np.array([[1., 2., np.nan],
              [3., 4., 5.],
              [5., 6., 7.]])
col_zscore(M).round(3)

array([[-1.225, -1.225,    nan],
       [ 0.   ,  0.   , -1.   ],
       [ 1.225,  1.225,  1.   ]])

In [48]:
# 4) Top-k indices (k=3) with stable ties
rng = np.random.default_rng(0)
v = rng.integers(0, 100, size=12)
idx_sorted = np.argsort(v, kind="stable")
top3_idx = idx_sorted[-3:][::-1]
v, top3_idx, v[top3_idx]

(array([85, 63, 51, 26, 30,  4,  7,  1, 17, 81, 64, 91], dtype=int64),
 array([11,  0,  9], dtype=int64),
 array([91, 85, 81], dtype=int64))

In [49]:
# 5) Stable softmax over axis=1
def softmax2d(X):
    X = np.asarray(X, dtype=float)
    Xs = X - X.max(axis=1, keepdims=True)
    ex = np.exp(Xs)
    return ex / ex.sum(axis=1, keepdims=True)

T = np.array([[1., 2., 3.],
              [3., 2., 1.],
              [0., 0., 0.]])
S = softmax2d(T)
S.round(4), np.allclose(S.sum(axis=1), 1.0)

(array([[0.09  , 0.2447, 0.6652],
        [0.6652, 0.2447, 0.09  ],
        [0.3333, 0.3333, 0.3333]]),
 True)